In [1]:
from utils import get_documents_from_folder
from document_processing import build_document_object, build_passages_objects

path_to_pdf_folder = "../documents"
path_to_text_folder = "../extracted"
pdf_documents = get_documents_from_folder(path_to_pdf_folder)
passages_objects = []
for pdf_document in pdf_documents:
    french_passages_objects, dutch_passages_objects, _, _ = build_passages_objects(
        pdf_document, path_to_pdf_folder, path_to_text_folder
    )
    passages_objects.extend(french_passages_objects)
    passages_objects.extend(dutch_passages_objects)


In [5]:
len(passages_objects)

320

In [12]:
from sentence_transformers import SentenceTransformer, LoggingHandler
from sentence_transformers import models, util, datasets, evaluation, losses
from torch.utils.data import DataLoader
import torch
# Define your sentence transformer model using CLS pooling
model_name = 'bert-base-uncased'
word_embedding_model = models.Transformer(model_name)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), 'cls')
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

# Define a list with sentences (1k - 100k sentences)
train_sentences = [passage.passage_text for passage in passages_objects]

# Create the special denoising dataset that adds noise on-the-fly
train_dataset = datasets.DenoisingAutoEncoderDataset(train_sentences)

# DataLoader to batch your data
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Use the denoising auto-encoder loss
train_loss = losses.DenoisingAutoEncoderLoss(model, decoder_name_or_path=model_name, tie_encoder_decoder=True)
#export to device
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
model.to(device)
train_loss.to(device)

When tie_encoder_decoder=True, the decoder_name_or_path will be invalid.
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.11.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.5.crossattention.self.query.bias', 'bert.encoder.layer.10.crossattention.self.query.bias', 'bert.encoder.layer.11.crossattention.self.key.weight', 'bert.encoder.layer.2.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.value.bias', 'bert.encoder.layer.10.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.3.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.9.crossattention.self.query.bias', 'bert.encoder.layer.10.crossattention.output.dense.bias', 'bert.encoder.layer.9.crossattention.output.dense.bias', 'bert.encoder.layer.5.crossattention.output.LayerNorm

DenoisingAutoEncoderLoss(
  (encoder): SentenceTransformer(
    (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
    (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  )
  (decoder): BertLMHeadModel(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=T

In [ ]:
# from torch.utils.tensorboard import SummaryWriter
# writer = SummaryWriter('runs/daeloss')
# Train the model
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=1,
    evaluation_steps=1000,
    warmup_steps=1000,
    output_path="output/daeloss",
    use_amp=True,
    optimizer_params={'lr': 2e-5}
)


/Users/senatlionnel/.conda/envs/devSenate/lib/python3.9/site-packages/torch/cuda/amp/grad_scaler.py:125: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/40 [00:00<?, ?it/s]

/Users/senatlionnel/.conda/envs/devSenate/lib/python3.9/site-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
